# Data Collection
<br>
Given the COVID-19 crisis, we will try to understand the health care capacity for India.
<br><br>
<i>The fight against COVID-19 is all about flattening the curve.</i>
<br>
<img src='https://thespinoff.co.nz/wp-content/uploads/2020/03/Covid-19-curves-graphic-social-v3.gif' alt='Flatten the curve' width=600 align='left'>

* **What data do we need to better understand the healthcare capacity of India?**
    - Discuss

Now that we understand what data we need to estimate the health care capacity for India. The logical next step is to find relevant data sources.
Unfortunately, this is not a straight forward process. There are multiple ways in which we can start searching for data sources, here are a few ways to start with:
1. Quick google search
2. Public [APIs](https://api.covid19india.org/)
3. Official websites, in this case [MoHFW](https://www.mohfw.gov.in/) and [ICMR](https://www.icmr.gov.in/)
4. Data groups working on similar space [datameet](http://datameet.org/)
5. Public feeds like twitter or facebook
6. [data.world](https://data.world/)

**Poll 1: Before we start, how much time do you think a Data Scientist spends in munging the data?**

## Pull Data from an API

Let us now pull the hospital beds data for **India**.     
[COVID 19 API List for India](https://api.rootnet.in/)

In [1]:
# Import libraries
import requests
import numpy as np
import pandas as pd
from pathlib import Path

pd.options.display.max_rows = 250
pd.options.display.max_columns = 50

# Set data path
DATA = Path('data')
!ls {DATA}

medical_college_list.csv  nhrr	pneumonia_2018.csv  Poll.docx


### Get data from the API

In [2]:
BED_URL = 'https://api.rootnet.in/covid19-in/hospitals/beds'

r = requests.get(BED_URL)
print(f'Status {r.status_code}')


Status 200


### Load it into a `pandas DataFrame`

In [3]:
api = r.json()

beds = pd.DataFrame(api['data']['regional'])
beds.head()

,state,ruralHospitals,ruralBeds,urbanHospitals,urbanBeds,totalHospitals,totalBeds,asOn
0,Andhra Pradesh,193,6480,65,16658,258,23138,2017-01-01T00:00:00.000Z
1,Arunachal Pradesh,208,2136,10,268,218,2404,2017-12-31T00:00:00.000Z
2,Assam,1176,10944,50,6198,1226,17142,2017-12-31T00:00:00.000Z
3,Bihar,930,6083,103,5936,1033,12019,2016-12-31T00:00:00.000Z
4,Chhattisgarh,169,5070,45,4342,214,9412,2016-01-01T00:00:00.000Z


### State wise bed count

In [4]:
(beds[:-1][['state', 'totalBeds']]
     .style
     .hide_index()
     .background_gradient(subset='totalBeds', cmap='YlGn'))

state,totalBeds
Andhra Pradesh,23138
Arunachal Pradesh,2404
Assam,17142
Bihar,12019
Chhattisgarh,9412
Goa,3013
Gujarat,32280
Haryana,11240
Himachal Pradesh,12399
Jammu & Kashmir,11651


### Exercise 1

1. Get data for hospital stats at a more granular level. Use the [Medical College API](https://api.rootnet.in/covid19-in/hospitals/medical-colleges)
2. Check the status code
3. Load the `medicalColleges` data into pandas DataFrame. (Hint: Check the structure of the response before loading it into the DataFrame)
4. Find top 5 states with minimum & maximum number of `hospitalBeds`
5. Did you notice any difference in number of hospital beds? Can you reason why?


In [5]:
# 1
COLLEGE_URL = 'https://api.rootnet.in/covid19-in/hospitals/medical-colleges'

# YOUR CODE GOES HERE

In [6]:
# 2

In [7]:
# 3

In [8]:
# 4

In [9]:
# 5

Great, now that we have **# of beds available** in each state, let us extract the **# of corona cases** per state. This will help us to better understand the shortage of beds in the coming future.

## Scrape Data from the WEB

We will be scraping the data from Worldometer or Ministry of Health & Family Welfare website.

- [MoHFW](https://www.mohfw.gov.in/)
- [Worldometer](https://www.worldometers.info/coronavirus/)

> BeautifulSoup - [Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

> Further reading
- https://do.co/2XzV5uT
- https://bit.ly/2A2axqo

### Get the *source* of the Worldometer or MoHFW webpage

In [10]:
MOHFW_URL = 'https://www.mohfw.gov.in/'
WORLDOMETER_URL= 'https://www.worldometers.info/coronavirus/'

In [11]:
india = pd.DataFrame(requests.get('https://www.mohfw.gov.in/data/datanew.json').json())
india.head()

,sno,state_name,active,positive,cured,death,new_active,new_positive,new_cured,new_death,state_code
0,2,Andaman and Nicobar Islands,403,636,226,7,484,734,242,8,35
1,1,Andhra Pradesh,72188,150209,76614,1407,74404,158764,82886,1474,28
2,3,Arunachal Pradesh,701,1673,969,3,699,1698,996,3,12
3,4,Assam,10183,41726,31442,101,10415,42904,32384,105,18
4,5,Bihar,18937,54240,34994,309,20306,57024,36389,329,10


In [12]:
r = requests.get(WORLDOMETER_URL)
print(f'Status: {r.status_code}')

Status: 200


### Extract the *table* from the *source*

In [13]:
# !conda install -y -c conda-forge beautifulsoup4 bs4

In [14]:
# Import BeautifulSoup
import bs4
from bs4 import BeautifulSoup as BS

page = BS(r.content, 'html.parser')
table = page.select('table#main_table_countries_today')[0]

### Look at the underlying structure of the *table*

In [15]:
table.tbody.tr.select('td')

[<td></td>,
 <td style="text-align:left;">
 <nobr>North America</nobr>
 </td>,
 <td>5,660,076</td>,
 <td>+7,902</td>,
 <td>222,207</td>,
 <td>+306</td>,
 <td>2,920,114</td>,
 <td>+5,166</td>,
 <td>2,517,755</td>,
 <td>26,024</td>,
 <td></td>,
 <td></td>,
 <td></td>,
 <td></td>,
 <td></td>,
 <td data-continent="North America" style="display:none;">North America</td>,
 <td>
 </td>,
 <td></td>,
 <td></td>]

### Extract *data* from the *table*

In [16]:
from typing import List

def extract_from_table(table: bs4.element.Tag) -> List:
    '''Extracts data from HTML table.
    
    Input:  bs4 *table*
    Return: List of all the values in the table
    '''
    data = list()
    header = list()
    
    # Extract the headers
    for row in table.select('thead tr'):
        header.append([head.text for head in row.select('th')])
    
    # Extract the rows
    for row in table.select('tbody tr'):
        data.append([col.text for col in row.select('td')])
        
    return header, data

header, data_table = extract_from_table(table)

In [17]:
header[0]

['#',
 'Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'NewRecovered',
 'ActiveCases',
 'Serious,Critical',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/\n1M pop\n',
 'Population',
 'Continent',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl']

In [18]:
data_table[0]

['',
 '\nNorth America\n',
 '5,660,076',
 '+7,902',
 '222,207',
 '+306',
 '2,920,114',
 '+5,166',
 '2,517,755',
 '26,024',
 '',
 '',
 '',
 '',
 '',
 'North America',
 '\n',
 '',
 '']

### Create a `pandas DataFrame` from the *data_table* & fix the `dtypes`

In [19]:
columns = ['idx', 'country', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_recovered', 
           'new_recovered', 'active_cases', 'critical', 'total_cases_per_million', 'deaths_per_million', 
           'total_tests', 'tests_per_million', 'population', 'continent', '1_case_every_X_ppl', '1_death_every_X_ppl', '1_test_every_X_ppl']
stats = pd.DataFrame(data_table[8:-8], columns=columns)

In [20]:
stats.head()

,idx,country,total_cases,new_cases,total_deaths,new_deaths,total_recovered,new_recovered,active_cases,critical,total_cases_per_million,deaths_per_million,total_tests,tests_per_million,population,continent,1_case_every_X_ppl,1_death_every_X_ppl,1_test_every_X_ppl
0,1,USA,"4,815,776","+2,129","158,376",+11,"2,380,561",+344,"2,276,839","18,623","14,541",478,"59,937,617","180,984","331,176,957",North America,69,"2,091",6
1,2,Brazil,"2,733,677",,"94,130",,"1,884,051",,"755,496","8,318","12,853",443,"13,096,132","61,573","212,694,204",South America,78,"2,260",16
2,3,India,"1,822,112","+17,410","38,400",+239,"1,200,303","+13,075","583,409","8,944","1,319",28,"20,202,858","14,627","1,381,196,835",Asia,758,"35,969",68
3,4,Russia,"856,264","+5,394","14,207",+79,"653,593","+3,420","188,464","2,300","5,867",97,"29,029,900","198,916","145,940,242",Europe,170,"10,272",5
4,5,South Africa,"511,485",,"8,366",,"347,227",,"155,892",539,"8,615",141,"3,036,779","51,147","59,373,395",Africa,116,"7,097",20


In [21]:
stats = stats.assign(**{
    'new_cases': pd.to_numeric(stats.new_cases.str.replace(r'[+,]', ''), downcast='integer', errors='coerce'),
    'new_deaths': pd.to_numeric(stats.new_deaths.str.replace(r'[+,]', ''), downcast='integer', errors='coerce'),
    'new_recovered': pd.to_numeric(stats.new_recovered.str.replace(r'[+,]', ''), downcast='integer', errors='coerce')
})

### Country wise new cases, deaths & recovery

In [22]:
(stats
     .filter(['country', 'new_cases', 'new_deaths', 'new_recovered'], axis=1)
     .fillna(0)
     .sort_values(by='new_cases', ascending=False)
     .style
         .format('{:,.0f}', subset=['new_cases', 'new_deaths', 'new_recovered'])
         .background_gradient(subset=['new_cases'], cmap='Oranges')
         .background_gradient(subset=['new_deaths'], cmap='Reds')
         .background_gradient(subset=['new_recovered'], cmap='Greens'))

,country,new_cases,new_deaths,new_recovered
2,India,"17,410",239,"13,075"
3,Russia,"5,394",79,"3,420"
5,Mexico,"4,853",274,"4,547"
24,Philippines,"3,226",45,274
20,Iraq,"2,735",66,"2,225"
10,Iran,"2,598",215,"2,126"
0,USA,"2,129",11,344
22,Indonesia,"1,679",66,"1,262"
29,Bolivia,"1,360",89,188
15,Bangladesh,"1,356",30,"1,066"


### Exercise 2

Scrape the top section of [MoHFW](https://www.mohfw.gov.in/) page, for helpline number, toll free number & email id. 

**Poll 2: What is unique about the section we are going to scrape in this page?**

In [23]:
mohfw = requests.get(MOHFW_URL)
page = BS(mohfw.content, 'html.parser')

In [24]:
# YOUR CODE GOES HERE

Ahh cool! We now have state level cases data & the # of beds available with us. But can we do better?  
Can we get data at District level instead? Let's try!

## Parse Data from PDF

We will be parsing the data from [National Health Profile (NHP)](https://www.cbhidghs.nic.in/index7.php?lang=1&level=0&linkid=1086&lid=1107&color=1) reports published by Central Bureau of Health Intelligence (CBHI) every year.  

You can download the PDF from here: [NHP 2019](https://github.com/srmsoumya/dsct/raw/master/data/dw/nhrr/NHRR2019.pdf) & save it in `data` directory

We will be using Camelot to parse PDF.  
[Camelot](https://camelot-py.readthedocs.io/en/master/)

### Extract *medical college data* from NHP 2019 report

> Page [270-282]

In [25]:
# !conda install -y -c conda-forge camelot-py (not working)
# !pip install camelot-py[cv]

In [26]:
import camelot

NHRR = DATA/'nhrr'/'NHRR2019.pdf'

In [27]:
med_clgs = camelot.read_pdf(str(NHRR), pages='270-282', flavor='lattice')

In [28]:
med_clgs[0].parsing_report

{'accuracy': 100.0, 'whitespace': 14.29, 'order': 1, 'page': 270}

In [29]:
med_clgs[0].df

,0,1,2,3,4,5,6
0,S. \nNo.,State/UT,Name of Medical College,City/Town,Govt/ \nPrivate,Admission \nCapacity,No. of \nbeds in \nAttached \nHospital
1,1,Andaman & \nNicobar Islands,Andaman & Nicobar Islands Insitute of Medical ...,Port Blair,Govt.,100,460
2,2,Andhra Pradesh,ACSR Government Medical College Nellore,Nellore,Govt.,150,750
3,3,,"All India Institute of Medical Sciences, Manga...",Vijaywada,Govt.,50,NA
4,4,,Alluri Sitaram Raju Academy of Medical Science...,Eluru,Trust,150,1070
5,5,,"Andhra Medical College, Visakhapatnam",Visakhapatnam,Govt.,200,2017
6,6,,Apollo Institute of Medical Sciences and Resea...,Chittoor,Society,150,NA
7,7,,"Dr. P.S.I. Medical College , Chinoutpalli",Chinoutpalli,Trust,150,398
8,8,,"Fathima Instt. of Medical Sciences,Kadapa",Kadapa,Trust,100,450
9,9,,Gayathri Vidya Parishad Institute of Health Ca...,Visakhapatnam,Society,150,NA


### Clean the table

In [30]:
def extract_table(df: pd.DataFrame) -> pd.DataFrame:
    '''Cleans the Dataframe'''
    df = df.copy()                                           # Work on a copy
    df.columns = df.iloc[0]                                  # Set Row 1 as the Column
    df.drop(df.index[0], inplace=True)               # Delete Row 1
    df.columns = [c.replace(' \n', '') for c in df.columns]  # Format column names
    df = df[df['S.No.'] != '']                               # Remove the total Rows
    df.set_index(keys='S.No.', inplace=True)                 # Set S.No as the index
    
    return df

med_clgs_df = pd.concat([extract_table(med_clgs[i].df) for i in range(13)])

In [31]:
# Fill the missing names in `State/UT` column, format the names
med_clgs_df['State/UT'] = med_clgs_df['State/UT'].replace(r'^\s*$', np.nan, regex=True)\
                                                 .ffill()\
                                                 .str.replace('\n', '')

In [32]:
med_clgs_df.head()

,State/UT,Name of Medical College,City/Town,Govt/Private,AdmissionCapacity,No. of beds in AttachedHospital
S.No.,,,,,,
1,Andaman & Nicobar Islands,Andaman & Nicobar Islands Insitute of Medical ...,Port Blair,Govt.,100,460
2,Andhra Pradesh,ACSR Government Medical College Nellore,Nellore,Govt.,150,750
3,Andhra Pradesh,"All India Institute of Medical Sciences, Manga...",Vijaywada,Govt.,50,NA
4,Andhra Pradesh,Alluri Sitaram Raju Academy of Medical Science...,Eluru,Trust,150,1070
5,Andhra Pradesh,"Andhra Medical College, Visakhapatnam",Visakhapatnam,Govt.,200,2017


### Save the data to a CSV file

In [33]:
med_clgs_df.to_csv(DATA/'medical_college_list.csv', index=False)

### *Exercise 3: Take home assignment

Extract **Pneumonia** data from NHP 2019 report
> Page: [139]

In [34]:
# YOUR CODE GOES HERE